In [34]:
pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [3]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [5]:
PINECONE_API_KEY = "3c244373-c13c-4b8b-b49f-bd47d2701f47"
PINECONE_API_ENV = "us-east-1-gcp-free"

In [6]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [62]:
extracted_data = load_pdf("data/")

In [63]:
type(extracted_data)

list

In [64]:
len(extracted_data)

5

In [65]:
extracted_data[0]

Document(page_content=" \n  Page | 1 \n \nE-commerce - Business models \nEvery business operates on business model of its own. Business model defines how a company \nruns its operations & generates revenue. Every viable organization is built on a sound business \nmodel. Selecting an ecommerce business model is a challenge, especially for beginners who \nhave little to no experience in the industry. If a business model is successfully executed, an \necommerce venture can become a significant source of income. Let us go through some most \nprevalent e-commerce business models in detail: \n1. B2B Business model \nA website following the B2B business model sells its products to an intermediate buyer who \nthen sells the product to the final customer. As an example, a wholesaler places an order from a \ncompany's website and after receiving the consignment, sells the end product to the final \ncustomer who comes to buy the product at one of its retail outlets.  \n \n \n  \n \n \n \nExample 

In [66]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50, chunk_overlap = 2)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [67]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 251


In [68]:
#download embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    

c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [81]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [82]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [71]:
import os

In [73]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'chatbot-2-oqs9n7z.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'chatbot-2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [74]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [90]:
index_name="chatbot-2"
from langchain_pinecone import PineconeVectorStore  
text_field = text_chunks 
vectorstore = PineconeVectorStore(  
    index, embeddings, text_field  
)  


In [91]:
query = "whAT IS business model?"  
X=vectorstore.similarity_search(  
    query,  # our search query  
    k=3  # return 3 most relevant docs  
) 

In [92]:
X

[]

In [80]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)  

 
index_name="chatbot-2"
index = pc.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 282}},
 'total_vector_count': 282}

In [59]:
#index = pinecone.Index(index_name)

In [58]:
# Define the embedding function
def embedding_function(query):
    return model.encode(query)

# Load the index into the Pinecone vector store
docsearch = Pinecone(index=index, embedding_function=embedding_function)

# Perform the similarity search
query = "What are Business?"
docs = docsearch.similarity_search(query, k=3)

# Print results
print("Result", docs)


AttributeError: 'Pinecone' object has no attribute 'similarity_search'

In [50]:
#If we already have an index we can load it like this
docsearch=Pinecone(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'startswith'

In [46]:
import pinecone
from pinecone import Pinecone,ServerlessSpec

In [50]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: list_indexes is no longer a top-level attribute of the pinecone package.

To use list_indexes, please create a client instance and call the method there instead.

Example:

    from pinecone import Pinecone
    
    pc = Pinecone(api_key='YOUR_API_KEY')

    index_name = "quickstart" # or your index name

    if index_name not in pc.list_indexes().names():
        # do something



In [36]:
query = "What is business model"
from pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_texts(texts = query, embedding=embeddings, index_name=index_name)

ImportError: cannot import name 'PineconeVectorStore' from 'pinecone' (c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\pinecone\__init__.py)

In [52]:
pip install langchain pinecone-client


In [2]:
from langchain_pinecone import PineconeVectorStore

ModuleNotFoundError: No module named 'langchain_pinecone'

In [1]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

c:\Users\hp\anaconda3\envs\chatbot2\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


ModuleNotFoundError: No module named 'langchain_pinecone'

In [55]:



docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is business model"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

In [1]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_texts(texts = text_chunks, embedding=embeddings, index_name=index_name)


c:\Users\hp\anaconda3\envs\bot3\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'text_chunks' is not defined

In [1]:
pip install langchain_pinecone

Note: you may need to restart the kernel to use updated packages.
